In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1"

In [2]:
wandb.init(project="tesi", name="robertaEnrichedLarge")  # args = TrainingArguments(


NameError: name 'wandb' is not defined

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("roberta-large")


/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("./robertaLargeInstanceOf/checkpoint-17500/")
model = AutoModelForQuestionAnswering.from_pretrained(
    "./robertaLargeInstanceOf/checkpoint-17500/",
    return_dict=False,
).to("cuda")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

file_path = "./train_converted.jsonl"
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [9]:
test = "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend 'Venite Ad Me Omnes'. Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."

In [11]:
print(test[514:])

 Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


In [2]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./train4.jsonl"
from datasets import Dataset

original_data = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        additional_question = "<additional> "
        try:
            for related in data_line["most_related"]:
                length = len(related)
                if len(related) == 4:
                    additional_question += related[2]
                    additional_question += " "
        except:
            None
        additional_question += "</additional>"
        data_line["context"] += additional_question
        
        original_data.append(data_line)
print(len(original_data))

7939


In [25]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./aida_train_instanceof.jsonl"
from datasets import Dataset

original_data_aida = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        # additional_question = "<additional> "
        # try:
        #     for related in data_line["most_related"]:
        #         length = len(related)
        #         if len(related) == 4:
        #             additional_question += related[2]
        #             additional_question += " "
        # except:
        #     None
        # additional_question += "</additional>"
        # data_line["context"] += additional_question
        
        original_data_aida.append(data_line)
print(len(original_data_aida))

18448


In [26]:
import random

# random.shuffle(original_data_aida)
# concat =original_data_aida[:int(len(original_data_aida)* 0.5)] + original_data
# random.shuffle(concat)
concat = [] + original_data_aida
print(len(concat))
train_original = concat[: int(len(concat) * 0.9)]
eval_original = concat[int(len(concat) * 0.9) :]
dataset = Dataset.from_dict(
    {
        "context": [item["question"] for item in train_original],
        "question": [item["context"] for item in train_original],
        "answers": [item["answers"] for item in train_original],
    }
)
dataset_ev = Dataset.from_dict(
    {
        "context": [item["question"] for item in eval_original],
        "question": [item["context"] for item in eval_original],
        "answers": [item["answers"] for item in eval_original],
    }
)

18448


/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [28]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("roberta-large")


def get_correct_alignement(context, answer):
    """Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here."""
    gold_text = answer["text"][0]

    start_idx = context.find(gold_text)
    end_idx = context.find("</ec>", start_idx)
    return start_idx, end_idx
    # if context[start_idx:end_idx] == gold_text:
    #     return start_idx, end_idx       # When the gold label position is good
    # elif context[start_idx-1:end_idx-1] == gold_text:
    #     return start_idx-1, end_idx-1   # When the gold label is off by one character
    # elif context[start_idx-2:end_idx-2] == gold_text:
    #     return start_idx-2, end_idx-2   # When the gold label is off by two character
    # else:
    #     raise ValueError()


# Tokenize our training dataset
def convert_to_features(example):
    try:
        # Tokenize contexts and questions (as pairs of inputs)

        input_pairs = [example["question"], example["context"]]
        encodings = tokenizer.encode_plus(
            input_pairs, pad_to_max_length=True, truncation="only_second"
        )
        context_encodings = tokenizer.encode_plus(example["context"])

        # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methodes.
        # this will give us the position of answer span in the context text

        start_idx, end_idx = get_correct_alignement(
            example["context"], example["answers"]
        )
        if end_idx != -1 and start_idx != -1:
            try:
                start_positions_context = context_encodings.char_to_token(start_idx)
                end_positions_context = context_encodings.char_to_token(end_idx)

                # here we will compute the start and end position of the answer in the whole example
                # as the example is encoded like this <s> question</s></s> context</s>
                # and we know the postion of the answer in the context
                # we can just find out the index of the sep token and then add that to position + 1 (+1 because there are two sep tokens)
                # this will give us the position of the answer span in whole example
                sep_idx = encodings["input_ids"].index(tokenizer.sep_token_id)
                start_positions = start_positions_context + sep_idx + 1
                end_positions = end_positions_context + sep_idx + 1
               
                encodings.update(
                    {
                        "start_positions": start_positions,
                        "ids": encodings["input_ids"],
                        "end_positions": end_positions,
                        "attention_mask": encodings["attention_mask"],
                        "targets": 1
                        if example["answers"]["text"][0] == "Not In Candidates"
                        else 0,
                    }
                )
                return encodings
            except Exception as e:
                print(e)
        else:
            print("qu")
            encodings.update(
                {
                    "start_positions": start_idx,
                    "ids": encodings["input_ids"],
                    "end_positions": end_idx,
                    "attention_mask": encodings["attention_mask"],
                    "targets": 1
                    if example["answers"]["text"][0] == "Not In Candidates"
                    else 0,
                }
            )
            return encodings
    except Exception as e:
        print("errors", e)

In [29]:
train_ds = dataset.map(convert_to_features)
eval_ds = dataset_ev.map(convert_to_features) 

Map:   0%|          | 0/16603 [00:00<?, ? examples/s]

/home/rub/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1216 > 512). Running this sequence through the model will result in indexing errors


qu
qu
qu
qu


Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

In [5]:
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    RobertaForQuestionAnswering,
)
from transformers import Trainer

import torch
from torch.nn import BCELoss


class CustomRobertaForQuestionAnswering(RobertaForQuestionAnswering):
    def __init__(self, config):
        super().__init__(config)
        self.additional_output1 = nn.Linear(1024, 1024)
        self.additional_output2 = nn.Linear(1024, 512)
        self.additional_output3 = nn.Linear(512, 512)
        self.additional_output4 = nn.Linear(512, 256)
        self.additional_output5 = nn.Linear(256, 256)
        self.additional_output6 = nn.Linear(256, 128)
        self.additional_output7 = nn.Linear(128, 128)
        self.additional_output8 = nn.Linear(128, 64)
        self.additional_output9 = nn.Linear(64, 64)
        self.additional_output10 = nn.Linear(64, 32)
        self.additional_output11 = nn.Linear(32, 32)
        self.additional_output12 = nn.Linear(32, 16)
        self.additional_output13 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

        self.additional_output1.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output2.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output3.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output4.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output5.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output6.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        if self.additional_output1.bias is not None:
            self.additional_output1.bias.data.zero_()
        if self.additional_output2.bias is not None:
            self.additional_output2.bias.data.zero_()
        if self.additional_output3.bias is not None:
            self.additional_output3.bias.data.zero_()
        if self.additional_output4.bias is not None:
            self.additional_output4.bias.data.zero_()
        if self.additional_output5.bias is not None:
            self.additional_output5.bias.data.zero_()
        if self.additional_output6.bias is not None:
            self.additional_output6.bias.data.zero_()
        if self.additional_output7.bias is not None:
            self.additional_output7.bias.data.zero_()
        if self.additional_output8.bias is not None:
            self.additional_output8.bias.data.zero_()
        if self.additional_output9.bias is not None:
            self.additional_output9.bias.data.zero_()
        if self.additional_output10.bias is not None:
            self.additional_output10.bias.data.zero_()
        if self.additional_output11.bias is not None:
            self.additional_output11.bias.data.zero_()
        if self.additional_output12.bias is not None:
            self.additional_output12.bias.data.zero_()
        if self.additional_output13.bias is not None:
            self.additional_output13.bias.data.zero_()
        self.additional_output1.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output2.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output3.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output4.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output5.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output6.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output7.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output8.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output9.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output10.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output11.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output12.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
        self.additional_output13.weight.data.normal_(
            mean=0.0, std=config.initializer_range
        )
       

    def forward(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        is_nil=None,
        end_positions=None,
    ):
        outputs = super().forward(
            input_ids,
            attention_mask,
            token_type_ids,
            position_ids,
            head_mask,
            inputs_embeds,
            start_positions,
            end_positions,
            output_hidden_states=True,
        )
        # print(outputs)
        hidden_states = outputs.hidden_states
        last_hidden_state = hidden_states[-1]
        cls_last_hidden_state = last_hidden_state[:, 0, :]
        additional_output = self.additional_output1(cls_last_hidden_state)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output2(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output3(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output4(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output5(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output6(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output7(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output8(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output9(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output10(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output11(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.additional_output12(additional_output)
        additional_output = self.relu(additional_output)
        additional_output = self.sigmoid(self.additional_output13(additional_output)).squeeze(-1)
        

        return (outputs, additional_output)


class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        is_nil = inputs.pop("is_nil").float()
        outputs, additional_output = model(**inputs)
        # Compute the original loss
        loss = outputs.loss.mean()
        # Compute the additional loss
        additional_loss = nn.functional.binary_cross_entropy(
            additional_output, is_nil, reduction="mean", reduce=True
        )

        # Combine the losses
        total_loss = loss + additional_loss
        return (total_loss, outputs) if return_outputs else total_loss

In [32]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

import wandb

# wandb.init(project="tesi", name="robertaAidaLargeNERNIL")  # args = TrainingArguments(
#     f"longformer-finetuned-squad",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

args = TrainingArguments(
    "robertaLargeInstanceOfBigBS",
    evaluation_strategy="steps",
    save_strategy="steps",
    report_to="wandb",
    load_best_model_at_end=False,
    logging_strategy="steps",
    logging_steps=20,
    eval_steps=20,
    save_steps=20,
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=100,
    num_train_epochs=1,
    weight_decay=0.02,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
!shutdown -h now 

In [8]:
trainer.save_model('./RoBertAAida')

Saving model checkpoint to ./RoBertAAida
Configuration saved in ./RoBertAAida/config.json
Model weights saved in ./RoBertAAida/pytorch_model.bin
tokenizer config file saved in ./RoBertAAida/tokenizer_config.json
Special tokens file saved in ./RoBertAAida/special_tokens_map.json


In [ ]:
torch.cuda.empty_cache()

In [59]:
dataset[0]

{'context': 'Give It All (Train song) </ec> Give It All (film) </ec> Give It All </ec>  NIL </ec>',
 'question': "t==Helen McNichol (Tania Raymonde), a high school senior, is in love with Stanford Prescott (Ryan Merriman), her jock boyfriend, and has decided to  [START_ENT] give it all [END_ENT] up to him. At least until she learns that her name is written in the high school football team's secret Bang Book and it is Stanford's j",
 'answers': {'answer_start': [75], 'text': ['NIL']}}

## Prediction and evaluation

In [2]:
def check_brackets(input_string):
    stack = []
    brackets = {"(": ")", "{": "}", "[": "]"}
    for char in input_string:
        if char in brackets.keys():
            stack.append(char)
        elif char in brackets.values():
            if not stack or brackets[stack.pop()] != char:
                return False
    return not stack


def process_answer(answer, candidates):
    if answer == "Not In Candidates":
        return answer
    else:
        modified_answer = answer.split("</ec>")[0]
        modified_answer = answer.split(": instance of ")[0]
        modified_answer = answer.split(": instance")[0]
        modified_answer = modified_answer.replace("<s>", "")
        modified_answer = modified_answer.replace("</s>", "")
        modified_answer = modified_answer.replace("<s", "")
        modified_answer = modified_answer.replace("</", "")
        modified_answer = modified_answer.replace(" ec", "")
        modified_answer = modified_answer.replace(">", "")
        modified_answer = modified_answer.strip()
        if not check_brackets(modified_answer):
            modified_answer = modified_answer.replace("(", "")
            modified_answer = modified_answer.replace(")", "")
        if modified_answer == "Not In Candidates":
            modified_answer = "NIL"
        if modified_answer == "":
            modified_answer = "Not In Candidates"
        if modified_answer not in candidates:
            modified_answer = "Not In Candidates"
        return modified_answer

In [3]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer_classfier = AutoTokenizer.from_pretrained("roberta-base")
classifier = RobertaForSequenceClassification.from_pretrained(
    "./robertaClassifier2/checkpoint-100/", return_dict=False, num_labels=2
).to("cuda:0")

In [31]:
import gc
import torch.nn.functional as F


def check_is_nil(input_pairs):
    encodings_c = tokenizer.encode_plus(
        input_pairs, return_tensors="pt", truncation="only_second"
    ).to("cuda")
    outs = classifier(
        encodings_c["input_ids"], attention_mask=encodings_c["attention_mask"]
    )
    outs = outs[0].cpu().detach().numpy()
    if outs[0][0] < outs[0][1]:
        return True
    else:
        return False


def make_prediction(data_entry, nil_prediction):
    with torch.no_grad():
        try:
            question = data_entry["input"]
            context = ""
            for item in data_entry["candidates"]:
                context += item + f" </ec> "
            if nil_prediction:
                context += "Not In Candidates </ec>"
            input_pairs = [question, context]
            encodings = tokenizer.encode_plus(
                input_pairs, return_tensors="pt", truncation="only_second"
            ).to("cuda")
            encodings_classifier = tokenizer_classfier.encode_plus(
                input_pairs, return_tensors="pt", truncation="only_second"
            ).to("cuda")
            is_nil = False
            if nil_prediction:
                is_nil = check_is_nil(input_pairs)

                if is_nil:
                    return {
                        "correct": data_entry["output"],
                        "non_processed": "Classified Not In Candidates",
                        "predicted": "Not In Candidates",
                        "input_phrase": question,
                        "scores": -1,
                        "candidates": context,
                    }
            answer = ""
            mean_score = 0

            start_scores, end_scores = model(
                encodings["input_ids"], attention_mask=encodings["attention_mask"]
            )
            start_scores = F.softmax(start_scores, dim=1)
            end_scores = F.softmax(end_scores, dim=1)
            # start_scores.to("cpu")
            # end_scores.to("cpu")
            start = torch.argmax(
                start_scores
            )  # Get the most likely beginning of answer with the argmax of the score
            end = (
                torch.argmax(end_scores) + 1
            )  # Get the most likely end of answer with the argmax of the score
            mean_score = 0
            try:
                mean_score = end_scores[0][end.item()] + start_scores[0][start.item()]
            except:
                None
            answer_tokens = encodings["input_ids"][0, start.item() : end.item() + 1]
            answer = ""

            answer = process_answer(tokenizer.decode(answer_tokens), context)

            del encodings
            del encodings_classifier
            if not is_nil:
                del start_scores
                del end_scores
                del start
                del end

            score = float(mean_score)
            gc.collect()
            torch.cuda.empty_cache()
            return {
                "correct": data_entry["output"],
                "non_processed": tokenizer.decode(answer_tokens)
                if not is_nil
                else None,
                "predicted": answer,
                "input_phrase": question,
                "scores": score,
                "candidates": context,
            }
        except Exception as e:
            print(e)
            return {
                "correct": data_entry["output"],
                "non_processed": None,
                "predicted": "Not In Candidates",
                "input_phrase": question,
                "scores": 0,
                "candidates": context,
            }

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("roberta-large")

model = AutoModelForQuestionAnswering.from_pretrained("./robertaAidaLargeNERNIL_SpecialNIL2/checkpoint-600").to("cuda")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [32]:
from tqdm import tqdm
import json
import torch

results = []
file_path = "./msnbc_nilled.jsonl"
dataset = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)

        dataset.append(data_line)

for item in tqdm(dataset):
    pred = make_prediction(item, True)
    results.append(pred)

  2%|▏         | 16/656 [00:01<00:36, 17.61it/s]

Truncation error: Sequence to truncate too short to respect the provided max_length


 60%|██████    | 395/656 [00:24<00:16, 15.44it/s]

Truncation error: Sequence to truncate too short to respect the provided max_length


100%|██████████| 656/656 [00:41<00:00, 15.81it/s]


In [37]:
correct = 0
correct_trace = []
wrong = []
correct_nil = 0
wrong_nil = 0
full_data = []
for result in tqdm(results):
    processed = result["predicted"]
    if processed == result["correct"][0]["answer"]:
        correct += 1
        correct_trace.append(result)
        if processed == "Not In Candidates":
            correct_nil += 1

    else:
        if result["correct"][0]["answer"] == "Not In Candidates":
            wrong_nil += 1
        wrong.append(result)
   

100%|██████████| 656/656 [00:00<00:00, 1068529.49it/s]


In [38]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.41615853658536583
accuracy in nil prediction is: 0.9496124031007752


- Base: 65.9%
- Large: 73.21%(enriched) - 74.65% (standard)
- Large EN: 76.66%
- large NER: 75.5%
- Base-NER: 66.35%

## MSNBC large
- No EN: 91.92%
- Large NIL-NER: 87.65% / 88.87%
- RobertaAidaLargeNER: 93% on base dataset
- RobertaAidaLargeNER finetuned on instance of: 92.98% - 92.37 on base dataset

## NIL EL
- NER + AIDA + Clustering  : 63.10%
- NER + AIDA + Clustering OpenAi  : 64.61%

## ace2004
- NO NER: 79.76

In [39]:
import pandas as pd

df = pd.DataFrame(wrong)
df_c = pd.DataFrame(correct_trace)

In [42]:
n = 20
offset = 0
df.iloc[offset * n : offset * n + n]

,correct,non_processed,predicted,input_phrase,scores,candidates
0,"[{'answer': 'Not In Candidates', 'provenance':...",</s>Celebrity Fit Club (U.S. TV series) </ec,Celebrity Fit Club (U.S. TV series),Timberlake Diaz reportedly break up Former N S...,0.999980,Celebrity Fit Club (U.S. TV series) </ec> Not ...
1,"[{'answer': 'Not In Candidates', 'provenance':...",</s>Celebrity Fit Club (U.S. TV series) </ec,Celebrity Fit Club (U.S. TV series),Timberlake Diaz reportedly break up Former N S...,0.999944,Celebrity Fit Club (U.S. TV series) </ec> Not ...
2,"[{'answer': 'Not In Candidates', 'provenance':...",</s>Iraqi Armed Forces </ec,Iraqi Armed Forces,execution that was leaked I can officially now...,0.999996,Iraqi Armed Forces </ec> Iraqi security forces...
3,"[{'answer': 'Not In Candidates', 'provenance':...",</s>New York </ec,New York,Barbara Walters stands by Rosie O Donnell View...,0.999996,New York </ec> New York (magazine) </ec> Unite...
4,"[{'answer': 'Not In Candidates', 'provenance':...","</s>Las Vegas–Paradise, NV MSA </ec","Las Vegas–Paradise, NV MSA",Muslim with U S family is held turned away Aut...,0.999996,"Las Vegas–Paradise, NV MSA </ec> Las Vegas Str..."
5,"[{'answer': 'Not In Candidates', 'provenance':...","> California State University, Bakersfield </ec","California State University, Bakersfield",Muslim with U S family is held turned away Aut...,0.999889,"Bakersfield, Vermont </ec> California State Un..."
6,"[{'answer': 'West Virginia', 'provenance': [{'...",Classified Not In Candidates,Not In Candidates,U S mines still not safe enough experts say On...,-1.000000,Not In Candidates </ec> Not In Candidates </ec>
7,"[{'answer': 'Not In Candidates', 'provenance':...",> Alabama Crimson Tide football </ec,Alabama Crimson Tide football,worth an estimated 32 million plus incentives ...,0.999997,Alabama </ec> Alabama (band) </ec> Alabama Cri...
8,"[{'answer': 'West Virginia', 'provenance': [{'...",Classified Not In Candidates,Not In Candidates,U S mines still not safe enough experts say On...,-1.000000,Not In Candidates </ec> Not In Candidates </ec>
9,"[{'answer': 'Not In Candidates', 'provenance':...",</s>Star </ec,Star,Timberlake Diaz reportedly break up Former N S...,0.617034,"Star </ec> Star (sailboat) </ec> Star, Idaho <..."


In [64]:
df.to_csv("./errorsNil.csv")
df_c.to_csv("./correctNil.csv")

wandb: Network error (ConnectionError), entering retry loop.


In [1]:
from datasets import load_dataset

dataset = load_dataset("strombergnlp/ipm_nel")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


ConnectionError: Couldn't reach http://www.derczynski.com/resources/ipm_nel.tar.gz (ConnectTimeout(MaxRetryError("HTTPConnectionPool(host='www.derczynski.com', port=80): Max retries exceeded with url: /resources/ipm_nel.tar.gz (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f654873f220>, 'Connection to www.derczynski.com timed out. (connect timeout=100)'))")))

In [ ]:
dataset